<h2><center>Introduction</center></h2>

Predicting the closing price of various assets is one of the first things every data scientist tries to do at least once in his career. The world of trading is fascinating, and poses immense challenges for us. The area of trading by algorithms has been gaining strength for some years, and this is the motivation of this work. Create a model that is able to predict, with reasonable accuracy, the closing price of the Bitcoin asset, in daily time, and using only the price data.

The model evaluation metric chosen by the percentage error of the entire historical series, and which must be below 5%.

<h1><center>Prepare Data</center></h1>

The objective of this project is to create a model that is able to predict the price of Bitcoin, at the close of any given day, with an acceptable margin of error (+/- 5%).
The data source used was a dataset available on Kaggle, which contains the daily quotation since 2014 for the asset in question.

In this notebook, I will present the treatment I performed to clear NaN's data, and load the entire dataframe into a Sqlite database for the next steps.

Download Data: 11/07/2021

Download link: https://www.kaggle.com/varpit94/bitcoin-data-updated-till-26jun2021

<h3>Packages Import<h3>

In [1]:
import pandas as pd
from sqlalchemy import create_engine

<h3>Opening file and create Dataframe</h3>

In [2]:
frame = pd.read_csv('./data/BTC-USD.csv', parse_dates=['Date'])
frame

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,2.105680e+07
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,3.448320e+07
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,3.791970e+07
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,3.686360e+07
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,2.658010e+07
...,...,...,...,...,...,...,...
2483,2021-07-05,35284.343750,35284.343750,33213.660156,33746.003906,33746.003906,2.672155e+10
2484,2021-07-06,33723.507813,35038.535156,33599.917969,34235.195313,34235.195313,2.650126e+10
2485,2021-07-07,34225.679688,34997.664063,33839.289063,33855.328125,33855.328125,2.479603e+10
2486,2021-07-08,33889.605469,33907.906250,32133.183594,32877.371094,32877.371094,2.991040e+10


<h3>Check rows and columns quantity</h3>

In [3]:
print('Dataframe number of rows: ', frame.shape[0])
print('Dataframe number of columns: ', frame.shape[1])

Dataframe number of rows:  2488
Dataframe number of columns:  7


<h3>Check for duplicates rows</h3>

To check if there were duplicate rows in the dataframe, we based on the dates. For this, we performed a count of the number of times the values present in the "Date" column appeared.

We did not find repeated dates in our data.

In [4]:
frame.Date.value_counts()[frame.Date.value_counts() > 1]

Series([], Name: Date, dtype: int64)

<h3>Check for missing values</h3>

We also checked for rows with missing values, regardless of whether the values were in just one column or the entire row.

We found 4 rows with missing values.

In order not to erase the lines and harm the time series, we created a function to impute data to the lines where there was missing data. The function that will impute the data calculates the moving average, accepting that the period is changed, and imputes these values to the missing data.

With this there will be no "holes" in our time series.

In [5]:
def fill_na(frame, period):
    """
    Fill NA values with rolling mean
    
    IN: 
    frame: dataframe with NA values
    period: rolling period to get mean
    OUT:
    frame: dataframe without NA values
    """
    df_mean = frame.rolling(period, min_periods=int(period*0.5)).mean()
    for ix in frame[frame.Open.isna()].index:
        frame.loc[ix, 'Open':'Volume'] = df_mean.loc[ix,:].to_list()
    return frame

In [6]:
print('Rows with missing value: ', frame[frame.Open.isna()].shape[0])
frame = fill_na(frame, 21)
print('Rows with missing value after function: ', frame[frame.Open.isna()].shape[0])

Rows with missing value:  4
Rows with missing value after function:  0


<h3>Remove column</h3>
The "Adj Close" column is identical to the "Close" column, so it will be excluded from our dataframe for not adding new information.

In [7]:
print('Rows with diferent values for columns Adj Close and Close: ', frame[(frame.Close) != (frame['Adj Close'])].shape[0])
frame.drop(columns=['Adj Close'], inplace=True)
frame.head()

Rows with diferent values for columns Adj Close and Close:  0


,Date,Open,High,Low,Close,Volume
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,21056800.0
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,34483200.0
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,37919700.0
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,36863600.0
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,26580100.0


<h3>Put all columns names with lower case</h3>

In [8]:
frame.columns = list(item.lower() for item in frame.columns)

<h3>Export Dataframe</h3>

To store the data for the next steps, let's load it into a Sqlite database. For this, we create a connection using Sqlalchemy, and we will use the Pandas "to_sql" function to load the table. If there is already a table with the same name in the database, it will be deleted and the data loaded.

In [9]:
# create database file and engine
engine = create_engine('sqlite:///database/database.db', echo=False)

In [10]:
# load data to sql
frame.to_sql(
    'historical',
    con=engine, 
    if_exists='replace'
)